# Inicialização de Pesos


## Carregando dados do google drive


In [ ]:
from google.colab import drive
drive.mount._DEBUG = False
drive.mount('/content/drive', force_remount=True)

In [ ]:
#Carregue os dados PKLotSegmented.tar.gz para seu Google Drive antes de executar
#Ajuste o path para apontar para o local do arquivo

!mkdir /content/datasets

!tar -xf "/content/drive/MyDrive/PKLotSegmented.tar.gz" -C "/content/datasets"
#Ou, se o .tar.gz já estiver carregado no seu ambiente:
!tar -xf "/content/PKLotSegmented.tar.gz" -C "/content/datasets"

!mv /content/datasets/Treino-UFPR04 /content/datasets/treino
!mv /content/datasets/Validação-UFPR05 /content/datasets/validacao
!mv /content/datasets/Teste-PUC /content/datasets/teste

In [ ]:
import math
import copy

import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

## Dataloaders

In [ ]:
img_dim = 32

data_transforms = {
    "treino": transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((img_dim, img_dim)),
        transforms.ToTensor()
    ]),
    "validacao": transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((img_dim, img_dim)),
        transforms.ToTensor()
    ]),
    "teste": transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((img_dim, img_dim)),
        transforms.ToTensor()
    ]),
}

sets = ["treino", "validacao", "teste"]

data_dir = "/content/datasets"
image_datasets = {x: datasets.ImageFolder(root=f'{data_dir}/{x}', transform=data_transforms[x])
                  for x in sets}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2)
               for x in sets}

dataset_sizes = {x: len(image_datasets[x]) for x in sets}
class_names = image_datasets['treino'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Conectado em um ambiente com", device)

## Definindo a arquitetura de uma CNN

In [ ]:
import torch.nn.functional as f
import torch.nn as nn
import importlib
import torch


#As imagens no dataset estão em escala de cinza
INPUT_SHAPE = (1, 32, 32)
NUM_CLASSES = 2

class custom_3_layer(nn.Module):
    def __init__(self):
        super(custom_3_layer, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=INPUT_SHAPE[0], out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=0)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(1600, 64)
        self.fc2 = nn.Linear(64, NUM_CLASSES)

        # Troque as linhas a seguir para obter inicializadores diferentes
        self.__initialize_weights_constant()
        #self.__initialize_weights_he()


    def convs(self, x):
        x = f.relu(self.conv1(x))
        x = self.pool1(x)
        x = f.relu(self.conv2(x))
        x = self.pool2(x)
        x = f.relu(self.conv3(x))
        return x


    def forward(self, x):
        x = self.convs(x)
        x = self.flatten(x)
        x = f.relu(self.fc1(x))
        x = self.fc2(x)
        return x


    # todos os kernels usando o mesmo conjunto de valores iniciais
    # Biases inicializados com 1
    def __initialize_weights_constant(self):
      kernel_values = torch.tensor([[-0.0820,  0.0865, -0.3165],
                                    [-0.1377, -0.0569, -0.2960],
                                    [-0.2373, -0.3300, -0.2369]])

      for module in self.modules():
            if isinstance(module, nn.Conv2d):
                nn.init.ones_(module.bias)
                with torch.no_grad():
                    for out_channel in range(module.out_channels):
                        for in_channel in range(module.in_channels):
                              module.weight[out_channel, in_channel, :, :] = kernel_values

    def __initialize_weights_he(self):
      for module in self.modules():
            if isinstance(module, nn.Conv2d):
                nn.init.zeros_(module.bias)
                torch.nn.init.kaiming_uniform_(module.weight, a=0, mode='fan_in', nonlinearity='relu')


## Imprimindo pesos

In [ ]:
#imprimir pesos
def print_weights(model):
  for param in model.parameters():
    print(param.data)

## Definido função de treino

In [ ]:
# Função para treinar o modelo
def train_model(model, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        print(f"Época {epoch+1}/{epochs}")
        print('-' * 10)

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders['treino']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes['treino']
        epoch_acc = running_corrects.double() / dataset_sizes['treino']

        print(f'Treino Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        print()
    return model

## Instanciando rede

In [ ]:
import torch.optim as optim

model = custom_3_layer()
model = model.to(device)

print_weights(model)

In [ ]:
# Função de loss e o otimizador
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Treinar o modelo
print("Treinando")
train_model(model, criterion, optimizer_ft, epochs=5)

# Avaliar o modelo no conjunto de teste
print("Avaliando no conjunto de testes")
model.eval()
test_corrects = 0

for inputs, labels in dataloaders['teste']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    test_corrects += torch.sum(preds == labels.data)

test_acc = test_corrects.double() / dataset_sizes['teste']
print(f'Test Acc: {test_acc:.4f}')

In [ ]:
print_weights(model)